In [4]:
# === Imports ===
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from joblib import load

# === Load embeddings and IDs ===

# Cancerous
emb_cancer = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_forw_can.npy")
ids_cancer = pd.read_csv("/home/azureuser/dna_sequencing/model_training/embeddings_ids.csv")["id"]
labels_cancer = np.ones(len(ids_cancer), dtype=int)  # label 1

# Non-cancerous
emb_noncan = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_forw_noncan.npy")
ids_noncan = pd.read_csv("/home/azureuser/dna_sequencing/model_training/embeddings_ids2.csv")["id"]
labels_noncan = np.zeros(len(ids_noncan), dtype=int)  # label 0

# === Combine all ===
X = np.vstack([emb_cancer, emb_noncan])
all_ids = pd.concat([ids_cancer, ids_noncan], ignore_index=True)
all_labels = np.concatenate([labels_cancer, labels_noncan])

# === Create and Shuffle DataFrame ===
df_combined = pd.DataFrame({
    "id": all_ids,
    "label": all_labels,
    "embedding": list(X)
}).sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

# === Convert for Model Input ===
X_all = np.vstack(df_combined["embedding"].values)
y_all = df_combined["label"].values

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.2, stratify=y_all, random_state=42
)

# === Load Models ===
model_non_fine = load("/home/azureuser/dna_sequencing/model_training/random_forest_dnabert_model.joblib")
model_fine = load("/home/azureuser/dna_sequencing/model_training/forw_final_randomforest_optuna_model.joblib")

# === Predict and Evaluate ===
print("=== Classification Report: Non-Fine-Tuned Random Forest ===")
y_pred_non_fine = model_non_fine.predict(X_test)
print(classification_report(y_test, y_pred_non_fine))

print("\n=== Classification Report: Fine-Tuned Random Forest ===")
y_pred_fine = model_fine.predict(X_test)
print(classification_report(y_test, y_pred_fine))

=== Classification Report: Non-Fine-Tuned Random Forest ===
              precision    recall  f1-score   support

           0       0.93      0.92      0.92    110077
           1       0.93      0.94      0.93    120649

    accuracy                           0.93    230726
   macro avg       0.93      0.93      0.93    230726
weighted avg       0.93      0.93      0.93    230726


=== Classification Report: Fine-Tuned Random Forest ===


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 161 out of 161 | elapsed:    3.4s finished


              precision    recall  f1-score   support

           0       0.97      0.98      0.97    110077
           1       0.98      0.97      0.98    120649

    accuracy                           0.97    230726
   macro avg       0.97      0.98      0.97    230726
weighted avg       0.98      0.97      0.97    230726



In [6]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score

# Add these after your classification reports
print(f"Non-Fine-Tuned Accuracy: {accuracy_score(y_test, y_pred_non_fine):.4f}")
print(f"Fine-Tuned Accuracy: {accuracy_score(y_test, y_pred_fine):.4f}")

print(f"Non-Fine-Tuned AUC: {roc_auc_score(y_test, y_pred_non_fine):.4f}")
print(f"Fine-Tuned AUC: {roc_auc_score(y_test, y_pred_fine):.4f}")

# Confusion matrices
print("\nConfusion Matrix - Non-Fine-Tuned:")
print(confusion_matrix(y_test, y_pred_non_fine))
print("\nConfusion Matrix - Fine-Tuned:")
print(confusion_matrix(y_test, y_pred_fine))

Non-Fine-Tuned Accuracy: 0.9273
Fine-Tuned Accuracy: 0.9750
Non-Fine-Tuned AUC: 0.9268
Fine-Tuned AUC: 0.9753

Confusion Matrix - Non-Fine-Tuned:
[[101049   9028]
 [  7756 112893]]

Confusion Matrix - Fine-Tuned:
[[108047   2030]
 [  3747 116902]]


In [7]:
# === Imports ===
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from joblib import load
from tensorflow.keras.models import load_model

# === Load embeddings and IDs ===

# Cancerous
emb_cancer = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_backw_can.npy")
ids_cancer = pd.read_csv("/home/azureuser/dna_sequencing/model_training/backw_can_embeddings_ids.csv")["id"]
labels_cancer = np.ones(len(ids_cancer), dtype=int)

# Non-cancerous
emb_noncan = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_backw_noncan.npy")
ids_noncan = pd.read_csv("/home/azureuser/dna_sequencing/model_training/backw_noncan_embeddings_ids.csv")["id"]
labels_noncan = np.zeros(len(ids_noncan), dtype=int)

# === Combine all ===
X = np.vstack([emb_cancer, emb_noncan])
all_ids = pd.concat([ids_cancer, ids_noncan], ignore_index=True)
all_labels = np.concatenate([labels_cancer, labels_noncan])

# === Create and Shuffle DataFrame ===
df_combined = pd.DataFrame({
    "id": all_ids,
    "label": all_labels,
    "embedding": list(X)
}).sample(frac=1, random_state=42).reset_index(drop=True)

# === Convert for Model Input ===
X_all = np.vstack(df_combined["embedding"].values)
y_all = df_combined["label"].values

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.2, stratify=y_all, random_state=42
)

# === Load Models ===
rf_model = load("/home/azureuser/dna_sequencing/model_training/backw_final_randomforest_optuna_model.joblib")
nn_model = load_model("/home/azureuser/dna_sequencing/model_training/backww_final_nn_model.keras")

# === Predict and Evaluate: Random Forest ===
print("=== Classification Report: Random Forest (Backward) ===")
y_pred_rf = rf_model.predict(X_test)
print(classification_report(y_test, y_pred_rf))

# === Predict and Evaluate: Neural Network ===
print("\n=== Classification Report: Neural Network (Backward) ===")
y_pred_nn = nn_model.predict(X_test, batch_size=256)
y_pred_nn_labels = (y_pred_nn > 0.5).astype(int).flatten()  # assuming binary classification
print(classification_report(y_test, y_pred_nn_labels))

=== Classification Report: Random Forest (Backward) ===


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    4.7s
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed:    5.1s finished


              precision    recall  f1-score   support

           0       1.00      0.99      1.00    109641
           1       0.99      1.00      1.00    120612

    accuracy                           1.00    230253
   macro avg       1.00      1.00      1.00    230253
weighted avg       1.00      1.00      1.00    230253


=== Classification Report: Neural Network (Backward) ===
900/900 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step
              precision    recall  f1-score   support

           0       0.90      0.85      0.87    109641
           1       0.87      0.91      0.89    120612

    accuracy                           0.88    230253
   macro avg       0.88      0.88      0.88    230253
weighted avg       0.88      0.88      0.88    230253



In [8]:
# Random Forest AUC (using probabilities if available)
if hasattr(rf_model, 'predict_proba'):
    y_proba_rf = rf_model.predict_proba(X_test)[:, 1]
    print(f"Random Forest AUC (probabilities): {roc_auc_score(y_test, y_proba_rf):.4f}")
else:
    print(f"Random Forest AUC (predictions): {roc_auc_score(y_test, y_pred_rf):.4f}")

print("\nConfusion Matrix - Random Forest:")
print(confusion_matrix(y_test, y_pred_rf))

# Neural Network AUC (using probabilities from predict)
y_proba_nn = y_pred_nn.flatten()  # NN predict gives probabilities
print(f"Neural Network AUC (probabilities): {roc_auc_score(y_test, y_proba_nn):.4f}")

print("\nConfusion Matrix - Neural Network:")
print(confusion_matrix(y_test, y_pred_nn_labels))

# === Model Comparison Summary ===
print("\n" + "="*50)
print("MODEL COMPARISON SUMMARY")
print("="*50)

rf_acc = accuracy_score(y_test, y_pred_rf)
nn_acc = accuracy_score(y_test, y_pred_nn_labels)

if hasattr(rf_model, 'predict_proba'):
    rf_auc = roc_auc_score(y_test, y_proba_rf)
else:
    rf_auc = roc_auc_score(y_test, y_pred_rf)

nn_auc = roc_auc_score(y_test, y_proba_nn)

print(f"Random Forest - Accuracy: {rf_acc:.4f}, AUC: {rf_auc:.4f}")
print(f"Best AUC: {'Random Forest' if rf_auc > nn_auc else 'Neural Network'}")

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    4.8s
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed:    5.1s finished


Random Forest AUC (probabilities): 1.0000

Confusion Matrix - Random Forest:
[[108861    780]
 [    32 120580]]
Neural Network AUC (probabilities): 0.9493

Confusion Matrix - Neural Network:
[[ 92815  16826]
 [ 10669 109943]]

MODEL COMPARISON SUMMARY
Random Forest - Accuracy: 0.9965, AUC: 1.0000
Best AUC: Random Forest


### Checking Data Leakage for Backward

In [10]:
# === Data Leakage Detection Script ===
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from joblib import load
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load your data (same as your original code) ===
# Cancerous
emb_cancer = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_backw_can.npy")
ids_cancer = pd.read_csv("/home/azureuser/dna_sequencing/model_training/backw_can_embeddings_ids.csv")["id"]
labels_cancer = np.ones(len(ids_cancer), dtype=int)

# Non-cancerous
emb_noncan = np.load("/home/azureuser/dna_sequencing/model_training/embeddings_backw_noncan.npy")
ids_noncan = pd.read_csv("/home/azureuser/dna_sequencing/model_training/backw_noncan_embeddings_ids.csv")["id"]
labels_noncan = np.zeros(len(ids_noncan), dtype=int)

# Combine all
X = np.vstack([emb_cancer, emb_noncan])
all_ids = pd.concat([ids_cancer, ids_noncan], ignore_index=True)
all_labels = np.concatenate([labels_cancer, labels_noncan])

# Create DataFrame
df_combined = pd.DataFrame({
    "id": all_ids,
    "label": all_labels,
    "embedding": list(X)
}).sample(frac=1, random_state=42).reset_index(drop=True)

X_all = np.vstack(df_combined["embedding"].values)
y_all = df_combined["label"].values

# Train-test split
X_train, X_test, y_train, y_test, train_ids, test_ids = train_test_split(
    X_all, y_all, df_combined["id"], test_size=0.2, stratify=y_all, random_state=42
)

print("="*60)
print("DATA LEAKAGE DETECTION ANALYSIS")
print("="*60)

# === 1. Check for Identical IDs Between Cancer and Non-Cancer ===
print("\n1. CHECKING FOR ID OVERLAP BETWEEN CANCER AND NON-CANCER DATASETS")
print("-" * 55)

cancer_ids_set = set(ids_cancer)
noncan_ids_set = set(ids_noncan)
id_overlap = cancer_ids_set.intersection(noncan_ids_set)

print(f"Cancer IDs: {len(cancer_ids_set)}")
print(f"Non-cancer IDs: {len(noncan_ids_set)}")
print(f"Overlapping IDs: {len(id_overlap)}")

if id_overlap:
    print("🚨 WARNING: Same IDs found in both cancer and non-cancer datasets!")
    print(f"First 10 overlapping IDs: {list(id_overlap)[:10]}")
    print("This is DEFINITE data leakage - same samples labeled as both cancer and non-cancer!")
else:
    print("✅ No ID overlap between cancer and non-cancer datasets")

# === 2. Check for Duplicate IDs Within Each Dataset ===
print("\n2. CHECKING FOR DUPLICATE IDs WITHIN DATASETS")
print("-" * 45)

cancer_duplicates = ids_cancer[ids_cancer.duplicated()]
noncan_duplicates = ids_noncan[ids_noncan.duplicated()]

print(f"Duplicate IDs in cancer dataset: {len(cancer_duplicates)}")
print(f"Duplicate IDs in non-cancer dataset: {len(noncan_duplicates)}")

if len(cancer_duplicates) > 0:
    print("🚨 WARNING: Duplicate IDs found in cancer dataset!")
    print(f"First 10 duplicates: {list(cancer_duplicates[:10])}")

if len(noncan_duplicates) > 0:
    print("🚨 WARNING: Duplicate IDs found in non-cancer dataset!")
    print(f"First 10 duplicates: {list(noncan_duplicates[:10])}")

if len(cancer_duplicates) == 0 and len(noncan_duplicates) == 0:
    print("✅ No duplicate IDs within individual datasets")

# === 3. Check Train-Test ID Overlap ===
print("\n3. CHECKING FOR TRAIN-TEST ID OVERLAP")
print("-" * 35)

train_ids_set = set(train_ids)
test_ids_set = set(test_ids)
train_test_overlap = train_ids_set.intersection(test_ids_set)

print(f"Training IDs: {len(train_ids_set)}")
print(f"Test IDs: {len(test_ids_set)}")
print(f"Overlapping IDs: {len(train_test_overlap)}")

if train_test_overlap:
    print("🚨 CRITICAL: Same IDs in both train and test sets!")
    print(f"Overlapping IDs: {list(train_test_overlap)[:10]}")
else:
    print("✅ No ID overlap between train and test sets")

# === 4. Check Patient-Level Leakage ===
print("\n4. CHECKING FOR PATIENT-LEVEL LEAKAGE")
print("-" * 35)

def extract_patient_id(seq_id):
    """Extract patient ID from sequence ID - adjust based on your ID format"""
    # Common patterns in biological data:
    # TCGA-XX-XXXX-XXX-XXX-XXXX-XX
    # Patient_001_Sample_A
    # SRR123456.1
    
    seq_id_str = str(seq_id)
    
    # TCGA format
    if seq_id_str.startswith('TCGA'):
        parts = seq_id_str.split('-')
        if len(parts) >= 3:
            return f"{parts[0]}-{parts[1]}-{parts[2]}"  # TCGA-XX-XXXX
    
    # Patient_Sample format
    if 'Patient' in seq_id_str or 'patient' in seq_id_str:
        parts = seq_id_str.split('_')
        if len(parts) >= 2:
            return f"{parts[0]}_{parts[1]}"
    
    # SRR format (sequence read archive)
    if seq_id_str.startswith('SRR') or seq_id_str.startswith('ERR'):
        return seq_id_str.split('.')[0]
    
    # Generic underscore/dash separation
    if '_' in seq_id_str:
        return seq_id_str.split('_')[0]
    elif '-' in seq_id_str:
        return seq_id_str.split('-')[0]
    
    # If no pattern matches, return first part or whole ID
    return seq_id_str.split()[0] if ' ' in seq_id_str else seq_id_str

# Show some examples of patient ID extraction
print("Sample ID -> Patient ID extraction:")
sample_ids = list(all_ids)[:5]
for sid in sample_ids:
    pid = extract_patient_id(sid)
    print(f"  {sid} -> {pid}")

# Extract patient IDs
train_patients = set([extract_patient_id(id_) for id_ in train_ids])
test_patients = set([extract_patient_id(id_) for id_ in test_ids])
patient_overlap = train_patients.intersection(test_patients)

print(f"\nUnique patients in training: {len(train_patients)}")
print(f"Unique patients in test: {len(test_patients)}")
print(f"Overlapping patients: {len(patient_overlap)}")

if patient_overlap:
    print("🚨 WARNING: Same patients in both train and test sets!")
    print(f"First 10 overlapping patients: {list(patient_overlap)[:10]}")
    print("This suggests patient-level data leakage!")
else:
    print("✅ No patient overlap between train and test sets")

# === 5. Embedding Similarity Check ===
print("\n5. CHECKING EMBEDDING SIMILARITY")
print("-" * 30)

# Sample a subset for similarity check (full comparison might be too expensive)
sample_size = min(1000, len(X_test), len(X_train))
test_sample_idx = np.random.choice(len(X_test), min(sample_size, len(X_test)), replace=False)
train_sample_idx = np.random.choice(len(X_train), min(sample_size, len(X_train)), replace=False)

X_test_sample = X_test[test_sample_idx]
X_train_sample = X_train[train_sample_idx]

print(f"Comparing {len(X_test_sample)} test samples with {len(X_train_sample)} train samples...")

# Calculate cosine similarity
similarities = cosine_similarity(X_test_sample, X_train_sample)
max_similarities = np.max(similarities, axis=1)

# Check for suspiciously high similarities
threshold_95 = 0.95
threshold_99 = 0.99

high_sim_95 = np.sum(max_similarities > threshold_95)
high_sim_99 = np.sum(max_similarities > threshold_99)

print(f"Test samples with >95% similarity to training: {high_sim_95}/{len(X_test_sample)} ({high_sim_95/len(X_test_sample)*100:.1f}%)")
print(f"Test samples with >99% similarity to training: {high_sim_99}/{len(X_test_sample)} ({high_sim_99/len(X_test_sample)*100:.1f}%)")
print(f"Max similarity found: {np.max(max_similarities):.6f}")
print(f"Mean max similarity: {np.mean(max_similarities):.6f}")
print(f"Std of max similarities: {np.std(max_similarities):.6f}")

if high_sim_99 > 0:
    print("🚨 CRITICAL: Very high embedding similarities detected!")
    print("This strongly suggests identical or near-identical sequences in train/test!")
elif high_sim_95 > len(X_test_sample) * 0.1:  # More than 10%
    print("🚨 WARNING: Many high embedding similarities detected!")
    print("This may indicate data leakage or very similar sequences!")
else:
    print("✅ Embedding similarities look reasonable")

# === 6. Performance Reality Check ===
print("\n6. PERFORMANCE REALITY CHECK")
print("-" * 28)

# Load models and make predictions
try:
    rf_model = load("/home/azureuser/dna_sequencing/model_training/backw_final_randomforest_optuna_model.joblib")
    nn_model = load_model("/home/azureuser/dna_sequencing/model_training/backww_final_nn_model.keras")
    
    # Random Forest predictions
    y_pred_rf = rf_model.predict(X_test)
    rf_acc = accuracy_score(y_test, y_pred_rf)
    
    # Neural Network predictions
    y_pred_nn = nn_model.predict(X_test, batch_size=256, verbose=0)
    y_pred_nn_labels = (y_pred_nn > 0.5).astype(int).flatten()
    nn_acc = accuracy_score(y_test, y_pred_nn_labels)
    
    print(f"Random Forest Accuracy: {rf_acc:.4f}")
    print(f"Neural Network Accuracy: {nn_acc:.4f}")
    
    # Flag suspiciously high performance
    if rf_acc > 0.99 or nn_acc > 0.99:
        print("🚨 CRITICAL: Near-perfect accuracy detected!")
        print("This is highly suspicious and likely indicates data leakage!")
    elif rf_acc > 0.95 or nn_acc > 0.95:
        print("🚨 WARNING: Very high accuracy detected!")
        print("Verify this is realistic for your specific domain and dataset")
    else:
        print("✅ Performance levels seem reasonable")
        
    # Cross-validation consistency check
    print("\n6b. CROSS-VALIDATION CONSISTENCY CHECK")
    print("-" * 35)
    
    try:
        cv_scores_rf = cross_val_score(rf_model, X_all, y_all, cv=5, scoring='accuracy')
        cv_mean_rf = cv_scores_rf.mean()
        cv_std_rf = cv_scores_rf.std()
        
        print(f"Random Forest:")
        print(f"  Single split accuracy: {rf_acc:.4f}")
        print(f"  CV mean accuracy: {cv_mean_rf:.4f} (+/- {cv_std_rf * 2:.4f})")
        
        if rf_acc > cv_mean_rf + 2*cv_std_rf:
            print("  🚨 WARNING: Single split much better than CV - possible leakage!")
        else:
            print("  ✅ Single split consistent with CV")
            
    except Exception as e:
        print(f"Could not perform CV check: {e}")
        
except Exception as e:
    print(f"Could not load models for performance check: {e}")

# === 7. Summary and Recommendations ===
print("\n" + "="*60)
print("SUMMARY AND RECOMMENDATIONS")
print("="*60)

leakage_flags = []

if id_overlap:
    leakage_flags.append("🚨 CRITICAL: Same IDs in cancer and non-cancer datasets")

if len(cancer_duplicates) > 0 or len(noncan_duplicates) > 0:
    leakage_flags.append("🚨 WARNING: Duplicate IDs within datasets")

if train_test_overlap:
    leakage_flags.append("🚨 CRITICAL: Same IDs in train and test sets")

if patient_overlap:
    leakage_flags.append("🚨 WARNING: Same patients in train and test sets")

if high_sim_99 > 0:
    leakage_flags.append("🚨 CRITICAL: Very high embedding similarities")

try:
    if 'rf_acc' in locals() and 'nn_acc' in locals():
        if rf_acc > 0.99 or nn_acc > 0.99:
            leakage_flags.append("🚨 CRITICAL: Near-perfect model performance")
except:
    pass

if leakage_flags:
    print("POTENTIAL DATA LEAKAGE DETECTED:")
    for flag in leakage_flags:
        print(f"  {flag}")
    
    print("\nRECOMMENDED ACTIONS:")
    print("1. Remove any duplicate or overlapping IDs")
    print("2. Ensure train-test split at patient level, not sample level")
    print("3. Verify that embeddings were generated independently")
    print("4. Re-evaluate models after fixing leakage issues")
    print("5. Consider using temporal splits if time information available")
    
else:
    print("✅ No obvious data leakage detected!")
    print("Your experimental setup appears to be sound.")

print("\nNote: This analysis covers common leakage patterns.")
print("Domain-specific leakage might require additional checks.")

DATA LEAKAGE DETECTION ANALYSIS

1. CHECKING FOR ID OVERLAP BETWEEN CANCER AND NON-CANCER DATASETS
-------------------------------------------------------
Cancer IDs: 603058
Non-cancer IDs: 548205
Overlapping IDs: 0
✅ No ID overlap between cancer and non-cancer datasets

2. CHECKING FOR DUPLICATE IDs WITHIN DATASETS
---------------------------------------------
Duplicate IDs in cancer dataset: 0
Duplicate IDs in non-cancer dataset: 0
✅ No duplicate IDs within individual datasets

3. CHECKING FOR TRAIN-TEST ID OVERLAP
-----------------------------------
Training IDs: 921010
Test IDs: 230253
Overlapping IDs: 0
✅ No ID overlap between train and test sets

4. CHECKING FOR PATIENT-LEVEL LEAKAGE
-----------------------------------
Sample ID -> Patient ID extraction:
  SRR5177930.6 -> SRR5177930
  SRR5177930.9 -> SRR5177930
  SRR5177930.11 -> SRR5177930
  SRR5177930.16 -> SRR5177930
  SRR5177930.20 -> SRR5177930

Unique patients in training: 2
Unique patients in test: 2
Overlapping patients: 

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    1.1s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    4.8s
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed:    5.2s finished


Random Forest Accuracy: 0.9965
Neural Network Accuracy: 0.8806
🚨 CRITICAL: Near-perfect accuracy detected!
This is highly suspicious and likely indicates data leakage!

6b. CROSS-VALIDATION CONSISTENCY CHECK
-----------------------------------


[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:  6.6min
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed: 30.3min
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed: 32.4min finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:    4.4s
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed:    4.7s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:  6.5min
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed: 30.5min
[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed: 32.6min finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.9s
[Parallel(n_jobs=7)]: Done 186 tasks      |

Random Forest:
  Single split accuracy: 0.9965
  CV mean accuracy: 0.8351 (+/- 0.0018)
  🚨 WARNING: Single split much better than CV - possible leakage!

SUMMARY AND RECOMMENDATIONS
POTENTIAL DATA LEAKAGE DETECTED:
  🚨 WARNING: Same patients in train and test sets
  🚨 CRITICAL: Near-perfect model performance

RECOMMENDED ACTIONS:
1. Remove any duplicate or overlapping IDs
2. Ensure train-test split at patient level, not sample level
3. Verify that embeddings were generated independently
4. Re-evaluate models after fixing leakage issues
5. Consider using temporal splits if time information available

Note: This analysis covers common leakage patterns.
Domain-specific leakage might require additional checks.


[Parallel(n_jobs=7)]: Done 200 out of 200 | elapsed:    4.7s finished
